In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from scipy.stats import mode
import csv
# import modelvis
from sklearn.ensemble import RandomForestClassifier
%matplotlib

Using matplotlib backend: MacOSX


## Frame 
Given building details, your task is to build a model that can predict the extent of damage that has been done to a building after an earthquake.

## ACQUIRE
train.csv, test.csv, building_ownership_usage.csv, building_structure.csv, sample_submission.csv

In [2]:
df_bd = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/train.csv')
df_bo = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/Building_Ownership_Use.csv')
df_str = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/building_structure.csv')

result = pd.merge(df_str, df_bo, on='building_id')
df = pd.merge(df_bd,result,on="building_id")

In [3]:
df_test = pd.read_csv('/Users/shreyaspr/Desktop/building_damage/test.csv')
df_test = pd.merge(df_test,result,on="building_id")

In [4]:
df.shape
#58 because it has the grade_damage which is predicted data to trrain the classifier

(631761, 58)

In [5]:
df_test.shape

(421175, 57)

## REFINE
Refine the data aquired damage_grade

In [6]:
grade = {'Grade 5': 5, 'Grade 4': 4, 'Grade 3': 3, 'Grade 2': 2, 'Grade 1': 1}
area = {'Both': 0, 'Building removed': 1, 'Exterior': 2, 'Not able to inspect': 3, 'Interior': 4}
land = {'Flat': 0, 'Moderate slope': 1, 'Steep slope': 2}
foundation = {'Mud mortar-Stone/Brick': 0, 'Bamboo/Timber': 1, 'Cement-Stone/Brick': 2, 'RC': 3, 'Other': 4}
roof = {'Bamboo/Timber-Light roof':0, 'Bamboo/Timber-Heavy roof':1, 'RCC/RB/RBC': 3}
grnd_floor = {'Mud':0, 'Brick/Stone':1, 'RC':2, 'Timber':3, 'Other':4}
other_floor = {'TImber/Bamboo-Mud': 0, 'Timber-Planck':1, 'Not applicable':2, 'RCC/RB/RBC':3}
pos = {'Not attached':0, 'Attached-1 side':1, 'Attached-2 side':2, 'Attached-3 side':3}
plan = {'Rectangular':0, 'Square':1, 'L-shape':2, 'Multi-projected':3, 'T-shape':4, 'Others':5, 'U-shape':6, 'Building with Central Courtyard':7, 
        'E-shape':8, 'H-shape':9}
condition = {'Damaged-Not used':0, 'Damaged-Repaired and used':1, 'Damaged-Used in risk':2, 'Damaged-Rubble unclear':3, 'Damaged-Rubble clear':4, 
             'Not damaged':5, 'Damaged-Rubble Clear-New building built':6, 'Covered by landslide':7}
legal = {'Private':0, 'Public':1, 'Institutional': 2, 'Other':3}

In [7]:
df.area_assesed = df.area_assesed.map(area.get)
df.land_surface_condition = df.land_surface_condition.map(land.get)
df.foundation_type = df.foundation_type.map(foundation.get)
df.roof_type = df.roof_type.map(roof.get)
df.ground_floor_type = df.ground_floor_type.map(grnd_floor.get)
df.other_floor_type = df.other_floor_type.map(other_floor.get)
df.position = df.position.map(pos.get)
df.plan_configuration = df.plan_configuration.map(plan.get)
df.condition_post_eq = df.condition_post_eq.map(condition.get)
df.legal_ownership_status = df.legal_ownership_status.map(legal.get)

df.damage_grade = df.damage_grade.map(grade.get)

In [8]:
df.dtypes

area_assesed                                int64
building_id                                object
damage_grade                                int64
district_id                                 int64
has_geotechnical_risk                     float64
has_geotechnical_risk_fault_crack           int64
has_geotechnical_risk_flood                 int64
has_geotechnical_risk_land_settlement       int64
has_geotechnical_risk_landslide             int64
has_geotechnical_risk_liquefaction          int64
has_geotechnical_risk_other                 int64
has_geotechnical_risk_rock_fall             int64
has_repair_started                        float64
vdcmun_id                                   int64
district_id_x                               int64
vdcmun_id_x                                 int64
ward_id_x                                   int64
count_floors_pre_eq                         int64
count_floors_post_eq                        int64
age_building                                int64


In [10]:
df.count_families.fillna(0, inplace=True)

In [22]:
df.isnull().sum()

area_assesed                                  0
building_id                                   0
damage_grade                                  0
district_id                                   0
has_geotechnical_risk                         0
has_geotechnical_risk_fault_crack             0
has_geotechnical_risk_flood                   0
has_geotechnical_risk_land_settlement         0
has_geotechnical_risk_landslide               0
has_geotechnical_risk_liquefaction            0
has_geotechnical_risk_other                   0
has_geotechnical_risk_rock_fall               0
has_repair_started                        33417
vdcmun_id                                     0
district_id_x                                 0
vdcmun_id_x                                   0
ward_id_x                                     0
count_floors_pre_eq                           0
count_floors_post_eq                          0
age_building                                  0
plinth_area_sq_ft                       

## Exploration
Exploring the best way to define the "has_repair_started" null values

In [12]:
df.has_geotechnical_risk_rock_fall.value_counts()

0    609742
1     22019
Name: has_geotechnical_risk_rock_fall, dtype: int64

In [35]:
df.has_repair_started[df.has_secondary_use_institution == 1] = 1

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df.has_repair_started[df.has_secondary_use_school == 1] = 1
df.has_repair_started[df.has_secondary_use_health_post == 1] = 1

In [36]:
df.has_repair_started[df.has_secondary_use_institution == 1].head(1)

2459    1.0
Name: has_repair_started, dtype: float64

In [34]:
df.iloc[2459]

area_assesed                                        0
building_id                               121a7c14acd
damage_grade                                        1
district_id                                        12
has_geotechnical_risk                               0
has_geotechnical_risk_fault_crack                   0
has_geotechnical_risk_flood                         0
has_geotechnical_risk_land_settlement               0
has_geotechnical_risk_landslide                     0
has_geotechnical_risk_liquefaction                  0
has_geotechnical_risk_other                         0
has_geotechnical_risk_rock_fall                     0
has_repair_started                                  0
vdcmun_id                                        1244
district_id_x                                      12
vdcmun_id_x                                      1244
ward_id_x                                      124406
count_floors_pre_eq                                 2
count_floors_post_eq        